In [ ]:
# Load GPT model
%run gpt-model.ipynb

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])
researchographs Ying fading
researchbold charming Reeves
Mean:
  tensor([-0.3596, -0.2606])
Variance :
  tensor([0.2015, 0.2673])
Norm. Mean:
  tensor([    -0.0000,      0.0000], grad_fn=<MeanBackward1>)
Norm. Variance :
  tensor([1.0000, 1.0000], grad_fn=<VarBackward0>)
tensor([[0.2685, 0.7413],
        [0.2738, 0.7564],
        [0.2668, 0.7366],
        [0.2618, 0.7218],
        [0.2712, 0.7495]], grad_fn=<MmBackward0>)
tensor([[-0.4927, -0.0791],
        [-0.4938, -0.0806],
        [-0.4924, -0.0851],
        [-0.4923, -0.0819],
        [-0.4928, -0.0853]], grad_fn=<MmBackward0>)
Self Attention V2 output: 
 tensor([[-0.4927, -0.0791],
        [-0.4938, -0.0806],
        [-0.4924, -0.0851],
        [-0.4923, -0.0819],
        [-0.4928, -0.0853]], grad_fn=<MmBackward0>)
Self Attention V1 output: 
 tensor([[-0.4927, -0.0791],
        [-0.4938, -0.0806],
        [-0.4924, -0.0851],
        [-0.4923, -0.0819],
        [

# Generating Text

Define a functionality that allows encoding and decoding text

In [15]:
import tiktoken
import torch

def textToTokenIds(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    return torch.tensor(encoded).unsqueeze(0)

def tokenIdsToText(tokens, tokenizer):
    formatted = tokens.squeeze(0).tolist()
    return tokenizer.decode(formatted)


In [24]:
# Test functionality to encode and decode text.

tokenizer = tiktoken.get_encoding("gpt2")

testText = "Hello world <|endoftext|>"
print("Original:", testText)

encoded = textToTokenIds(testText, tokenizer)
print("Encoded: ", encoded)

decoded = tokenIdsToText(encoded, tokenizer)
print("Decoded: ", decoded)


del tokenizer, testText, encoded, decoded
# generateText(model, textToTokenIds(testText, tokenizer), 10, cfg["contextSize"] )

Original: Hello world <|endoftext|>
Encoded:  tensor([[15496,   995,   220, 50256]])
Decoded:  Hello world <|endoftext|>


Test functionality to run GPT model to generate text

In [34]:
import torch

tokenizer = tiktoken.get_encoding("gpt2")
maxNewTokens = 10
testText = "Hello world"

# Run model
encodedOutput = generateText(model, textToTokenIds(testText, tokenizer), maxNewTokens, cfg["contextLength"] )
print("Encoded output:\n ", encodedOutput)

print("Encoded generated text:\n ", tokenIdsToText(encodedOutput, tokenizer))

del tokenizer, maxNewTokens, encodedOutput

Encoded output:
  tensor([[15496,   995, 16621, 37792,  9096, 29641, 33062, 32260, 37431,   815,
         38790,  8833]])
Encoded generated text:
  Hello world expressing basilheim bots dungeons productions decon shouldjri occurs


## Calculating the text generation loss

The loss is needed to backpropagate the mistakes.

We can use the cross entropy loss, which tries to minimize the difference between two distributions: the model output distribution and the input text distribution (i.e. for evaluation or training). 

To do this, we first transform the output logits of the model into a probability distribution, using the softmax function. Then, we compute the cross entropy loss.

#### Cross Entropy

The cross entropy is defined as the negative log likelihood of the probability. 
- The closer to `1` the predicted probability is, the lower the loss. 
- Likewise, the predicted ouptut closer to `0`, the higher the loss.

We provide batches of inputs to the model, thus we will compute the average cross entropy loss per batch.